# Competition 3. Link prediction

## Challenge Overview
The dataset contains publications that is described by a feature vector of the paper description. The dataset can be represented as a directed graph where nodes are publications and edges are citations. The random part of edges are dropped and also disconnected pairs of nodes are selected in the same amount as dropped edges. Let us denote dropped edges by label 1 and additionally selected pairs by label 0.

The dataset is represented by 3 files:
- node_feat.txt contains description of the papers in the format: _<32d feature vector>_
-train_edges.txt contains existing edges in the format:
_\<node id\> \<node id\>_
-unlabeled_edges.txt contains unlabeled pairs of nodes in the format:
_\<node id\> \<node id\>_

Your task is to predict labels for unlabeled pairs of nodes: 0 — disconnected, 1 — connected.

## Evaluation Criteria
Here are balanced classes, so the usual accuracy metric is used:

**Accuracy = True predictions / All predictions**

The baselines are calculated as follows: 
1. Sample negative edges. Concatenate node features into edge features. Train Gradient boosting on edges features. Calculate score.
2. Create a graph on existing edges. Train Laplacian Eigenmaps. Sample negative edges. Concatenate node embeddings into edge embeddings. Train Gradient boosting on edge embeddings. Calculate score.
3. Create a graph using existing edges. Sample negative edges. Train a neural network consisting of GCN for node emedding and MLP for link prediction that maps Hadamart product edge embeddings into probability of edge existing. Calculate score.

Baseline for grade 4: beat a minimum score

Baseline for grade 6: beat a mean score

## Решение

In [1]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html


In [2]:
import copy
import itertools
import requests

import dgl
import dgl.function as fn
import networkx as nx
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv, GATConv, SAGEConv
from sklearn.decomposition import TruncatedSVD    
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Скачаем данные и проанализируем их

In [3]:
for name in ('node_feat.txt', 'train_edges.txt', 'unlabeled_edges.txt'):
    url = 'https://raw.githubusercontent.com/netspractice/network-science/main/datasets/lp_comp/' + name
    open(name, 'wb').write(requests.get(url).content)

In [4]:
train_edges = np.loadtxt('train_edges.txt', dtype=np.int64)
node_feat = np.loadtxt('node_feat.txt', dtype=np.float64)
unlabeled_edges = np.loadtxt('unlabeled_edges.txt', dtype=np.int64)

In [5]:
train_edges.shape, node_feat.shape, unlabeled_edges.shape

((14322, 2), (12588, 32), (44014, 2))

In [6]:
train_edges.max(), unlabeled_edges.max()

(12587, 12587)

В графе 12588 вершин (от 0 до 12587)

In [7]:
NUM_NODES = 12588

Будем строить стекинг градиентных бустингов с разными способами создания эмбедингов ребер (Adamar, mean, concat, L1, L2, weighted_L1, weighted_L2) из эмбедингов вершин, которые построим как конкатенацию фичей вершин с ее эмбедингом через SVD разложении матрицы смежности графа.

In [8]:
seed_array = (0, 1, 42)
predictions = {
    'weighted_L2': [],
    'weighted_L1': [],
    'Adamar': [],
    'concat': [],
    'L1': [],
    'L2': [],
    'mean': []
}

In [9]:
def SVD_features_GBC(train_edges=train_edges,
                     unlabeled_edges=unlabeled_edges,
                     test_size=0.1,
                     seed=42,
                     node_feat=node_feat,
                     n_components=128,
                     n_estimators=1000,
                     emb_edges_type='Adamar',
                     emb_A=None,
                     G=None):
                  
    assert emb_edges_type in (
        'Adamar', 'concat', 'L1', 'L2',
        'mean', 'weighted_L1', 'weighted_L2'
        ), "Incorrect emb_edges_type"

    np.random.seed(seed)

    if emb_A is None:
        G = nx.Graph()
        G.add_nodes_from(np.arange(NUM_NODES))
        G.add_edges_from(train_edges)
        A = nx.to_numpy_array(G)
        emb_A = TruncatedSVD(n_components=n_components).fit_transform(A)

    u, v = train_edges[:, 0], train_edges[:, 1]
    neg_v = np.random.randint(NUM_NODES, size=len(train_edges))
    
    emb = np.concatenate([node_feat, emb_A], axis=1)

    if emb_edges_type == 'Adamar':
        pos_X = emb[u] * emb[v]
        neg_X = emb[u] * emb[neg_v]
        X_test = emb[unlabeled_edges[:, 0]] * emb[unlabeled_edges[:, 1]]
    elif emb_edges_type == 'concat':
        pos_X = np.concatenate([emb[u], emb[v]], axis=1)
        neg_X = np.concatenate([emb[u], emb[neg_v]], axis=1)
        X_test = np.concatenate([emb[unlabeled_edges[:, 0]], emb[unlabeled_edges[:, 1]]], axis=1)
    elif emb_edges_type == 'L1':
        pos_X = np.abs(emb[u] - emb[v])
        neg_X = np.abs(emb[u] - emb[neg_v])
        X_test = np.abs(emb[unlabeled_edges[:, 0]] - emb[unlabeled_edges[:, 1]])
    elif emb_edges_type == 'L2':
        pos_X = (emb[u] - emb[v]) ** 2
        neg_X = (emb[u] - emb[neg_v]) ** 2
        X_test = (emb[unlabeled_edges[:, 0]] - emb[unlabeled_edges[:, 1]]) ** 2
    elif emb_edges_type == 'mean':
        pos_X = (emb[u] + emb[v]) / 2
        neg_X = (emb[u] + emb[neg_v]) / 2
        X_test = (emb[unlabeled_edges[:, 0]] + emb[unlabeled_edges[:, 1]]) / 2
    elif emb_edges_type == 'weighted_L1':
        pos_X, neg_X, X_test = [], [], []
        for u_node, v_node, neg_v_node in zip(u, v, neg_v):
            u_neigh = (emb[[u_node] + list(G.neighbors(u_node))]).mean(axis=0)
            v_neigh = (emb[[v_node] + list(G.neighbors(v_node))]).mean(axis=0)
            neg_v_neigh = (emb[[neg_v_node] + list(G.neighbors(neg_v_node))]).mean(axis=0)
            pos_X.append(np.abs(u_neigh - v_neigh))
            neg_X.append(np.abs(u_neigh - neg_v_neigh))
        for u_node, v_node in unlabeled_edges:
            u_neigh = (emb[[u_node] + list(G.neighbors(u_node))]).mean(axis=0)
            v_neigh = (emb[[v_node] + list(G.neighbors(v_node))]).mean(axis=0)
            X_test.append(np.abs(u_neigh - v_neigh))
        pos_X, neg_X, X_test = np.array(pos_X), np.array(neg_X), np.array(X_test)
    elif emb_edges_type == 'weighted_L2':
        pos_X, neg_X, X_test = [], [], []
        for u_node, v_node, neg_v_node in zip(u, v, neg_v):
            u_neigh = (emb[[u_node] + list(G.neighbors(u_node))]).mean(axis=0)
            v_neigh = (emb[[v_node] + list(G.neighbors(v_node))]).mean(axis=0)
            neg_v_neigh = (emb[[neg_v_node] + list(G.neighbors(neg_v_node))]).mean(axis=0)
            pos_X.append((u_neigh - v_neigh) ** 2)
            neg_X.append((u_neigh - neg_v_neigh) ** 2)
        for u_node, v_node in unlabeled_edges:
            u_neigh = (emb[[u_node] + list(G.neighbors(u_node))]).mean(axis=0)
            v_neigh = (emb[[v_node] + list(G.neighbors(v_node))]).mean(axis=0)
            X_test.append((u_neigh - v_neigh) ** 2)
        pos_X, neg_X, X_test = np.array(pos_X), np.array(neg_X), np.array(X_test)
            
    X = np.concatenate([pos_X, neg_X])
    y = np.concatenate([np.ones(pos_X.shape[0]), np.zeros(neg_X.shape[0])])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=seed)
    
    gbc = XGBClassifier(n_estimators=n_estimators, random_state=seed).fit(X_train, y_train)
    val_accuracy = accuracy_score(gbc.predict(X_val), y_val)
    test_prediction = gbc.predict(X_test)

    return val_accuracy, test_prediction

In [10]:
G = nx.Graph()
G.add_nodes_from(np.arange(NUM_NODES))
G.add_edges_from(train_edges)
A = nx.to_numpy_array(G)
emb_A = TruncatedSVD(n_components=256).fit_transform(A)

In [11]:
for seed in seed_array:
    for emb_edges_type in predictions.keys():
        accuracy, test_prediction = SVD_features_GBC(
            emb_edges_type=emb_edges_type,
            emb_A=emb_A,
            seed=seed,
            G=G,
            n_estimators=1000,
            test_size=0.01)
        print(f'seed = {seed}, \t type = {emb_edges_type}, \t val_accuracy = {accuracy}')
        predictions[emb_edges_type].append(test_prediction)

seed = 0, 	 type = weighted_L2, 	 val_accuracy = 0.9790940766550522
seed = 0, 	 type = weighted_L1, 	 val_accuracy = 0.9790940766550522
seed = 0, 	 type = Adamar, 	 val_accuracy = 0.9616724738675958
seed = 0, 	 type = concat, 	 val_accuracy = 0.9825783972125436
seed = 0, 	 type = L1, 	 val_accuracy = 0.867595818815331
seed = 0, 	 type = L2, 	 val_accuracy = 0.8536585365853658
seed = 0, 	 type = mean, 	 val_accuracy = 0.8641114982578397
seed = 1, 	 type = weighted_L2, 	 val_accuracy = 0.9790940766550522
seed = 1, 	 type = weighted_L1, 	 val_accuracy = 0.9825783972125436
seed = 1, 	 type = Adamar, 	 val_accuracy = 0.9721254355400697
seed = 1, 	 type = concat, 	 val_accuracy = 0.975609756097561
seed = 1, 	 type = L1, 	 val_accuracy = 0.8571428571428571
seed = 1, 	 type = L2, 	 val_accuracy = 0.8571428571428571
seed = 1, 	 type = mean, 	 val_accuracy = 0.8362369337979094
seed = 42, 	 type = weighted_L2, 	 val_accuracy = 0.9651567944250871
seed = 42, 	 type = weighted_L1, 	 val_accuracy = 0

In [12]:
predictions

{'Adamar': [array([0., 0., 0., ..., 1., 0., 0.]),
  array([0., 0., 0., ..., 1., 0., 0.]),
  array([0., 0., 0., ..., 1., 0., 1.])],
 'L1': [array([0., 0., 0., ..., 0., 0., 1.]),
  array([0., 0., 0., ..., 1., 0., 1.]),
  array([0., 0., 0., ..., 1., 0., 1.])],
 'L2': [array([0., 0., 0., ..., 0., 0., 1.]),
  array([0., 0., 0., ..., 1., 0., 1.]),
  array([0., 0., 0., ..., 1., 0., 1.])],
 'concat': [array([0., 0., 0., ..., 1., 0., 0.]),
  array([0., 0., 0., ..., 1., 0., 0.]),
  array([0., 0., 0., ..., 1., 0., 0.])],
 'mean': [array([0., 0., 0., ..., 1., 0., 1.]),
  array([0., 0., 0., ..., 1., 0., 1.]),
  array([0., 0., 0., ..., 1., 0., 1.])],
 'weighted_L1': [array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 1.]),
  array([0., 0., 0., ..., 0., 0., 1.])],
 'weighted_L2': [array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 1.]),
  array([0., 0., 0., ..., 0., 0., 1.])]}

In [13]:
all_pred = []
for name, pr in predictions.items():
    for arr in pr:
        all_pred.append(arr)

In [14]:
all_pred

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 1.]),
 array([0., 0., 0., ..., 0., 0., 1.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 1.]),
 array([0., 0., 0., ..., 0., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 1.]),
 array([0., 0., 0., ..., 0., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 1.]),
 array([0., 0., 0., ..., 1., 0., 1.])]

In [15]:
gbc_mean_predict = np.array(np.array(all_pred).mean(axis=0) > 0.5, dtype=np.int64)
gbc_mean_predict.sum()

10614

In [16]:
gbc_mean_predict = np.array(np.array(all_pred).mean(axis=0) > 0, dtype=np.int64)
gbc_mean_predict.sum()

26741

In [17]:
all_pred = []
for name, pr in predictions.items():
    if name in ('Adamar', 'concat', 'weighted_L1', 'weighted_L2'):
        for arr in pr:
            all_pred.append(arr)

In [18]:
gbc_mean_predict = np.array(np.array(all_pred).mean(axis=0) > 0.5, dtype=np.int64)
gbc_mean_predict.sum()

7722

Будем строить стэкинг по такому принципу: если хоть ОДНА модель посчитала, что ребро в графе есть, значит считаем что это ребро есть

In [19]:
gbc_mean_predict = np.array(np.array(all_pred).mean(axis=0) > 0, dtype=np.int64)
gbc_mean_predict.sum()

18930

In [20]:
np.savetxt('submit_gbc.txt', gbc_mean_predict, fmt='%i', delimiter='\n')

Скор на тестовых данных: 0.8041

Можно было бы в стекинг добавить нейронные сети, однако не удалось добиться на них качества больше чем 60%. Если подкрутить гиперпараметры или как-то по другому проводить само обучение, скорее всего качество бы повысилось. Ниже находятся неудачные эксперименты с ними

In [21]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats):
        super().__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

class GAT(nn.Module):
    def __init__(self, in_feats, h_feats):
        super().__init__()
        self.conv1 = GATConv(in_feats, h_feats, 8, feat_drop=0.2)
        self.conv2 = GATConv(h_feats * 8, h_feats, 1)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = h.view(-1, h.size(1) * h.size(2))
        h = F.elu(h)
        h = self.conv2(g, h)
        return h.squeeze()

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return g.edata['score'][:, 0]

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

In [34]:
def NN(train_edges=train_edges,
       unlabeled_edges=unlabeled_edges,
       test_size=0.1,
       seed=42,
       num_epochs=1000,
       lr=3e-4,
       print_train=True,
       name_model='GCN'):
  
    np.random.seed(seed)
    torch.manual_seed(seed)

    g = dgl.DGLGraph().to(device)
    g.add_edges(train_edges[:, 0], train_edges[:, 1])
    g.ndata['feat'] = torch.Tensor(node_feat)

    u, v = g.edges()

    eids = np.arange(g.number_of_edges())
    eids = np.random.permutation(eids)
    test_size = int(len(eids) * test_size)
    train_size = g.number_of_edges() - test_size
    test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
    train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

    neg_v = np.random.randint(NUM_NODES, size=len(train_edges))

    neg_eids = np.random.choice(len(neg_v), g.number_of_edges())
    test_neg_u, test_neg_v = test_pos_u, neg_v[neg_eids[:test_size]]
    train_neg_u, train_neg_v = train_pos_u, neg_v[neg_eids[test_size:]]

    train_g = dgl.remove_edges(g, eids[:test_size]).to(device)
    train_g = dgl.add_self_loop(train_g)

    train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes()).to(device)
    train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes()).to(device)

    test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes()).to(device)
    test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes()).to(device)

    if name_model == 'GCN':
        model = GCN(train_g.ndata['feat'].shape[1], 32)
    elif name_model == 'GAT':
        model = GAT(train_g.ndata['feat'].shape[1], 32)
    elif name_model == 'GraphSAGE':
        model = GraphSAGE(train_g.ndata['feat'].shape[1], 32)

    #pred = MLPPredictor(8)
    pred = DotPredictor()
    optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200, 400, 500, 600, 800], gamma=0.5)
    all_logits = []

    best_model = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for e in range(num_epochs):
        h = model(train_g, train_g.ndata['feat'])
        pos_score = pred(train_pos_g, h)
        neg_score = pred(train_neg_g, h)
        loss = compute_loss(pos_score, neg_score)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            pos_score = pred(test_pos_g, h)
            neg_score = pred(test_neg_g, h)
            loss_test = compute_loss(pos_score, neg_score)
            if loss_test < best_loss:
                best_model = copy.deepcopy(model.state_dict())
                best_loss = loss_test
           
        if e % (num_epochs // 20) == 0 and print_train:
            print('In epoch {}, loss: {}, test_loss: {}'.format(e, loss, loss_test))

    model.load_state_dict(best_model)
    test_g = dgl.DGLGraph().to(device)
    test_g.add_edges(unlabeled_edges[:, 0], unlabeled_edges[:, 1])
    test_g.ndata['feat'] = torch.Tensor(node_feat)
    test_g_ = dgl.add_self_loop(test_g).to(device)
    h = model(test_g_, test_g_.ndata['feat'])
    pos_score = pred(test_g, h)
    test_prediction = np.array(torch.sigmoid(pos_score) >= 0.5, dtype=np.int64)

    return test_prediction

In [35]:
NN()

In epoch 0, loss: 0.6820926666259766, test_loss: 0.7170300483703613
In epoch 50, loss: 0.6568458080291748, test_loss: 0.6699837446212769
In epoch 100, loss: 0.6359355449676514, test_loss: 0.6556169986724854
In epoch 150, loss: 0.6174577474594116, test_loss: 0.6488596200942993
In epoch 200, loss: 0.6015124917030334, test_loss: 0.6379255056381226
In epoch 250, loss: 0.5922828912734985, test_loss: 0.6305621266365051
In epoch 300, loss: 0.5833888053894043, test_loss: 0.6242129802703857
In epoch 350, loss: 0.5757639408111572, test_loss: 0.6199895143508911
In epoch 400, loss: 0.5694897174835205, test_loss: 0.617525577545166
In epoch 450, loss: 0.5667153000831604, test_loss: 0.616649329662323
In epoch 500, loss: 0.5640900731086731, test_loss: 0.6159230470657349
In epoch 550, loss: 0.5628214478492737, test_loss: 0.6156011819839478
In epoch 600, loss: 0.5615806579589844, test_loss: 0.6153140068054199
In epoch 650, loss: 0.5609689950942993, test_loss: 0.6151817440986633
In epoch 700, loss: 0.560

array([1, 1, 1, ..., 1, 1, 1])

In [41]:
seed_array = (0, 1, 42)
NN_predictions = {
    'GCN': [],
    'GAT': [],
    'GraphSAGE': [],
}

In [42]:
for seed in seed_array:
    for name_model in NN_predictions.keys():
        test_prediction = NN(
            seed=seed,
            name_model=name_model,
            print_train=False,
            test_size=0.01)
        print(f'seed = {seed}, \t type = {name_model}')
        NN_predictions[name_model].append(test_prediction)

seed = 0, 	 type = GCN
seed = 0, 	 type = GAT
seed = 0, 	 type = GraphSAGE
seed = 1, 	 type = GCN
seed = 1, 	 type = GAT
seed = 1, 	 type = GraphSAGE
seed = 42, 	 type = GCN
seed = 42, 	 type = GAT
seed = 42, 	 type = GraphSAGE


In [43]:
all_pred = []
for name, pr in NN_predictions.items():
    for arr in pr:
        all_pred.append(arr)

In [45]:
nn_mean_predict = np.array(np.array(all_pred).mean(axis=0) > 0.5, dtype=np.int64)
nn_mean_predict.sum()

40206

Слегка ужесточим трешхолд

In [53]:
nn_mean_predict = np.array(np.array(all_pred).mean(axis=0) > 0.9, dtype=np.int64)
nn_mean_predict.sum()

19704

In [54]:
np.savetxt('submit_nn.txt', nn_mean_predict, fmt='%i', delimiter='\n')